### Spark Demonstration

This is a demo of connecting Jupyter to Spark 2.0.0.

In [1]:
val sparkHome = "/root/spark-2.0.0-preview-bin-hadoop2.7"
val sparkAssembly = s"$sparkHome/lib/spark-assembly-2.0.0-preview.jar"
val sparkMaster = "spark://localhost:7077"
val sparkVersion = "2.0.0-preview"
val scalaVersion = scala.util.Properties.versionNumberString

sparkHome: String = "/Users/zack.angelo/Downloads/spark-2.0.0-preview"
sparkAssembly: String = "/Users/zack.angelo/Downloads/spark-2.0.0-preview/lib/spark-assembly-2.0.0-preview.jar"
sparkMaster: String = "spark://AUS-MBP-ANGELO:7077"
sparkVersion: String = "2.0.0-preview"
scalaVersion: String = "2.11.8"

In [10]:
classpath.add(
    "com.github.alexarchambault.ammonium" % s"spark_2.0.0-preview_2.11.8" % "0.4.0-SNAPSHOT",
    "com.databricks" % "spark-csv_2.11" % "1.4.0",
    "org.apache.spark" %% "spark-mllib" % "2.0.0-preview"
)

16 new artifact(s)


16 new artifacts in macro
16 new artifacts in runtime
16 new artifacts in compile


In [3]:
@transient val Spark = new ammonite.spark.Spark

Spark.sparkConf
    .setMaster("spark://localhost:7077")
    .setAppName("ammonium-spark")
    .set("spark.home", sparkHome)

Spark.sc

log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/06/09 08:22:12 INFO Spark$SparkContext: Running Spark version 2.0.0-preview
16/06/09 08:22:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/06/09 08:22:12 INFO SecurityManager: Changing view acls to: zack.angelo
16/06/09 08:22:12 INFO SecurityManager: Changing modify acls to: zack.angelo
16/06/09 08:22:12 INFO SecurityManager: Changing view acls groups to: 
16/06/09 08:22:12 INFO SecurityManager: Changing modify acls groups to: 
16/06/09 08:22:12 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(zack.angelo); groups with view permissions: Set(

Spark: ammonite.spark.Spark = Spark
res2_1: org.apache.spark.SparkConf = org.apache.spark.SparkConf@5ad08203
res2_2: org.apache.spark.SparkContext = SparkContext

In [11]:
import Spark.{ sparkConf, sc, sqlContext } //Imports must be in separate cell, at the top
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, VectorAssembler, StringIndexer}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import Spark.{ sparkConf, sc, sqlContext }
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, VectorAssembler, StringIndexer}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

In [5]:
val trainFile = sc.textFile("titanic/train.csv")

trainFile: org.apache.spark.rdd.RDD[String] = titanic/train.csv MapPartitionsRDD[1] at textFile at Main.scala:25

In [6]:
val trainData = trainFile.map(_.split(","))

trainData: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at Main.scala:25

In [7]:
def loadData(
  trainFile: String,
  testFile: String,
  sqlContext: org.apache.spark.sql.SQLContext
): (org.apache.spark.sql.DataFrame, org.apache.spark.sql.DataFrame) = {
  val nullable = true
  val schemaArray = Array(
    StructField("PassengerId", IntegerType, nullable),
    StructField("Survived", IntegerType, nullable),
    StructField("Pclass", IntegerType, nullable),
    StructField("Name", StringType, nullable),
    StructField("Sex", StringType, nullable),
    StructField("Age", FloatType, nullable),
    StructField("SibSp", IntegerType, nullable),
    StructField("Parch", IntegerType, nullable),
    StructField("Ticket", StringType, nullable),
    StructField("Fare", FloatType, nullable),
    StructField("Cabin", StringType, nullable),
    StructField("Embarked", StringType, nullable)
  )

  val trainSchema = StructType(schemaArray)
  val testSchema = StructType(schemaArray.filter(p => p.name != "Survived"))

  val csvFormat = "com.databricks.spark.csv"

  val trainDF = sqlContext.read
  .format(csvFormat)
  .option("header", "true")
  .schema(trainSchema)
  .load(trainFile)

  val testDF = sqlContext.read
  .format(csvFormat)
  .option("header", "true")
  .schema(testSchema)
  .load(testFile)

  (trainDF, testDF)
}

defined function loadData

In [8]:
val (trainDf,testDf) = loadData("titanic/train.csv", "titanic/test.csv", sqlContext)

trainDf: DataFrame = [PassengerId: int, Survived: int ... 10 more fields]
testDf: DataFrame = [PassengerId: int, Pclass: int ... 9 more fields]

In [9]:
trainDf.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|     |       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [11]:
trainDf.groupByKey(col("Sex"))

: 

In [12]:
trainDf.select("PassengerId", "Survived").show(10)

: 